<a href="https://colab.research.google.com/github/zcwisc/GB657/blob/main/Module_4_ImageClassificationViaCNNLab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab: Image Data and Image Classification via CNN

In this lab, we will consider a second image classification problem on an alternative dataset: The Fashion MNIST dataset. Here, we predict what fashion item is shown on a given image.


The dataset is fairly analogous to the MNIST data -- it's also 28 by 28 pixel images, although in this case the images are of fashion items and the labels correspond to different fashion items -- here is the label	description:
- 0	T-shirt/top
- 1	Trouser
- 2	Pullover
- 3	Dress
- 4	Coat
- 5	Sandal
- 6	Shirt
- 7	Sneaker
- 8	Bag
- 9	Ankle boot

The problem, obviously, is to predict what kind of fashion item is on the image.

## Load packages and Data

Let's load some packages:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import random

from sklearn.metrics import confusion_matrix

from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from keras.datasets import fashion_mnist
from keras.utils import to_categorical

And let's take a look at the data:

In [ ]:
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

...and at a few of the items:

In [ ]:
image_index = 5
print(y_train[image_index])
plt.imshow(x_train[image_index], cmap='Greys')
plt.show()

...so this is a pullover.

In [ ]:
image_index = 42
print(y_train[image_index])
plt.imshow(x_train[image_index], cmap='Greys')
plt.show()

So this is an ankle boot.

## Data Formatting

As before, let's set a random seed and let's format the data for modeling:

In [ ]:
np.random.seed(42)

num_classes = 10

y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

## Feed-forward Neural Net

Let's start by using a very similar neural net as we did in the previous application. We start by reshaping the inputs to simple vectors:

In [ ]:
x_train_trad = x_train.reshape(x_train.shape[0], 784)
x_test_trad = x_test.reshape(x_test.shape[0], 784)

x_train_trad = x_train_trad / 255
x_test_trad = x_test_trad/ 255

And let's build a feed-forward model, as we did before. Again we use the soft-max function as the output layer for this multi-class problem, and we use 'categorical_crossentropy' as the (multi-class) loss function:

In [ ]:
model = Sequential()
model.add(Dense(50, input_shape=(784, ), activation='relu', name='dense_1'))
model.add(Dense(25, activation='relu', name='dense_2'))
model.add(Dense(10, activation='softmax', name='dense_output'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Let's train it using 20 epochs:

In [ ]:
history = model.fit(x_train_trad, y_train, epochs=20, validation_data=(x_test_trad, y_test))

So, again, we notice that after around 15 eopochs the validation fit doesn't seem to improve much. And, the acuracy is lower than 90%. So, it appears that the neural net is not working super well here. Let's try...

## Convolutional Neural Nets



To run the convolutional neural net, we use the native 2-dimensional tensor format:

In [ ]:
img_rows, img_cols = 28, 28

x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)

And let's try buidiling a simple convolutional neural net:

In [ ]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(img_rows, img_cols, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(img_rows/2, img_cols/2, 1)))
model.add(Conv2D(16, kernel_size=(3, 3), activation='relu', input_shape=(img_rows/2, img_cols/2, 1)))
model.add(Flatten())
model.add(Dense(24, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy',optimizer='adam', metrics=['accuracy'])

In [ ]:
model.summary()

So this seems to have few similar number of parameters as our neural net above. Let's train (to be fair, I experimented a little bit to get this):

In [ ]:
history = model.fit(x_train, y_train, epochs=20, validation_data=(x_test, y_test))

So, it definitely takes longer to train. But, the accuracy definitely improves, though maybe not as drastically as we would hope. We may have to add more complexity to get to high accuracy, yet that requires a good amount of experimentation. As we indicated training these neural nets is part art aabd part science.

Let's check out the predictions. Here we are going to generate a multi-class confusion matrix:

In [ ]:
y_pred_prob = model.predict(x_test)
y_pred = np.argmax(y_pred_prob, axis=1)
y_true = np.argmax(y_test, axis=1)

conf_matrix = confusion_matrix(y_true, y_pred)
conf_matrix = confusion_matrix(y_true, y_pred, labels=list(range(num_classes)))
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt='g', cmap='Blues', xticklabels=list(range(num_classes)), yticklabels=list(range(num_classes)))
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

It is maybe not surpirisng that shirt and t-shirt present a challenge. Let's look at a few predictions:

In [ ]:
def plot_digit(image, digit, plt, i):
    plt.subplot(4, 5, i + 1)
    plt.imshow(image, cmap=plt.get_cmap('gray'))
    plt.title(f"Digit: {digit}")
    plt.xticks([])
    plt.yticks([])

random.seed(13)

plt.figure(figsize=(16, 10))
for i in range(20):
    image = random.choice(x_test).squeeze()
    digit = np.argmax(model.predict(image.reshape((1, 28, 28, 1)))[0], axis=-1)
    plot_digit(image, digit, plt, i)

plt.show()

0 T-shirt/top
1 Trouser
2 Pullover
3 Dress
4 Coat
5 Sandal
6 Shirt
7 Sneaker
8 Bag
9 Ankle boot